Модель нейронной сети, которая позволяет проводить распознование рукописных цифр. 

Модель выполнена по книге *Тарик Рашид - Создаем нейронную сеть* (https://www.ozon.ru/context/detail/id/141796497/)


In [ ]:
import numpy as np
# For SIGMA() function
import scipy.special

class neuralNetwork:
    
    def __init__(self, input_noes, hidden_nodes, out_nodes, learning_rate):
        self.inodes = input_noes
        self.hnodes = hidden_nodes
        self.onodes = out_nodes
        
        self.lr = learning_rate
        
        # Set weights between input and hidden layers
        # Задаются как выборка из нормального распределения с центром в нуле, и мат. ожиданием равным
        # 1. / (корень квадратный из количества узлов)
        self.wih = np.random.normal(0.0, pow(self.hnodes, -0.5), (self.hnodes, self.inodes))
        self.who = np.random.normal(0.0, pow(self.onodes, -0.5), (self.onodes, self.hnodes))
        
        # Activation function
        self.act_func = lambda x: scipy.special.expit(x)
    
    # NW train function
    def train(self, input_list, target_list):
        inputs = np.array(input_list, ndmin = 2).T
        targets = np.array(target_list, ndmin = 2).T
        
        # Caluclate output of hidden layer
        hidden_input = np.dot(self.wih, inputs)
        hidden_output = self.act_func(hidden_input)
        
        # Calculate output of output layers
        out_input = np.dot(self.who, hidden_output)
        out_output = self.act_func(out_input)
        
        # Calculate error of output layer
        out_errors = targets - out_output
        
        # Reverse error correction on output->hidden layer
        hidden_errors = np.dot(self.who.T, out_errors)
        self.who += self.lr * np.dot( (out_errors * out_output * (1.0 - out_output)), np.transpose(hidden_output) )
        
        # Reverse error correction on hidden->input layer
        self.wih += self.lr * np.dot( (hidden_errors * hidden_output * (1.0 - hidden_output)), np.transpose(inputs) )
        
    
    # Calculate output results based on choosen 'input_list' data
    def query(self, inputs_list):
        inputs = np.array(inputs_list, ndmin = 2).T
        
        hidden_input = np.dot(self.wih, inputs)
        hidden_output = self.act_func(hidden_input)
        
        out_input = np.dot(self.who, hidden_output)
        out_output = self.act_func(out_input)
        
        return out_output

In [131]:
import os

input_nodes = 784
hidden_nodes = 100
output_nodes = 10

learning_rate = 0.2

# Number of repetition for NW on single dataset
epoch_number = 5

train_file_path = os.path.join(os.getcwd(), 'mnist_dataset', 'mnist_train.csv')

if __name__ == '__main__':
    nw = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)
    
    # Prepare trainig data
    trainig_data_file = open(train_file_path, 'r')
    train_data_list = trainig_data_file.readlines()
    trainig_data_file.close()
    
    for i in range(epoch_number):
        for record in train_data_list:
            all_values = record.split(',')
            # Create data for input nodes
            # Convert from [0,255] range to [0.01, 1.0]
            input_data = (np.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01

            # Create output targets
            # If output is 5 -> [0.01, 0.01, 0.01, 0.01, 0.99, 0.01, 0.01, 0.01, 0.01,]
            output_data = np.zeros(output_nodes) + 0.01
            output_data[int(all_values[0])] = 0.99

            nw.train(input_data, output_data)

In [132]:
import matplotlib.pyplot as plt

test_file_path = os.path.join(os.getcwd(), 'mnist_dataset', 'mnist_test.csv')

test_data_file = open(test_file_path, 'r')
test_data_list = test_data_file.readlines()
test_data_file.close()

scores = []

if __name__ == '__main__':
    
    for record in test_data_list:
        test_values = record.split(',')
        test_input = (np.asfarray(test_values[1:]) / 255.0 * 0.99) + 0.01
        expected_value = int(test_values[0])
        #print('Expected value: {0}'.format(expected_value))
        #plt.imshow(test_input.reshape(28,28), cmap='Greys', interpolation='None')
        #plt.show()

        train_values = nw.query(test_input)
        
        if expected_value == np.argmax(train_values):
            scores.append(1)
        else:
            scores.append(0)
    
    scores = np.asarray(scores)
    print('NW efficiency: {0}'.format(float(np.sum(scores)) / float(scores.size)))
    

NW efficiency: 0.962
